In [1]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os

#print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
#model_name="nvidia/Llama3-ChatQA-1.5-8B"

model_name="../model/------llm-models-hf-2ja1_part_inst_all_lr_1e-5"
model_name="../X_merge/merged_models/0507test"
model_name="../../llm/models/hf/step41300",
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-4"
model_name="../model/------llm-models-hf-step41300_inst_all-parquet_lr_1e-5"
model_name="../model/------llm-models-hf-step41300_inst_all_5000-parquet_lr_1e-5"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

tokens=[5092, 272, 1045, 2850, 327, 8,11267, 269, 564, 298, 2]
tokenizer.decode(tokens,)

'### 応答:\n はい、可能です</s>'

In [3]:
t_list=[
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nああ",
"\n\n### 応答:\naaa",
"1\n\n### 応答:\naaa",
"あ\n\n### 応答:\naaa",
"」\n\n### 応答:\naaa",
]
t_list=[
    "",
    "a \n a",
"a\n   ",
"\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nああ",
"\n\n### 指示:\naaa",
"1\n\n### 指示:\naaa",
"あ\n\n### 指示:\naaa",
"」\n\n### 指示:\naaa",
]
for t in t_list:
    print(tokenizer.encode(t,add_special_tokens=False),tokenizer.tokenize(t))


[] []
[355, 272, 8, 355] ['▁a', '▁', '\n', '▁a']
[355, 8, 3223] ['▁a', '\n', '▁▁▁']
[272, 8, 8, 48979, 272, 3994, 327, 8] ['▁', '\n', '\n', '###', '▁', '指示', ':', '\n']
[272, 8, 8, 48979, 272, 3994, 327, 8, 292] ['▁', '\n', '\n', '###', '▁', '指示', ':', '\n', '「']
[272, 8, 8, 48979, 272, 3994, 327, 8, 2721] ['▁', '\n', '\n', '###', '▁', '指示', ':', '\n', 'ああ']
[272, 8, 8, 48979, 272, 3994, 327, 8, 55841] ['▁', '\n', '\n', '###', '▁', '指示', ':', '\n', 'aaa']
[272, 279, 8, 8, 48979, 272, 3994, 327, 8, 55841] ['▁', '1', '\n', '\n', '###', '▁', '指示', ':', '\n', 'aaa']
[4543, 8, 8, 48979, 272, 3994, 327, 8, 55841] ['▁あ', '\n', '\n', '###', '▁', '指示', ':', '\n', 'aaa']
[272, 285, 8, 8, 48979, 272, 3994, 327, 8, 55841] ['▁', '」', '\n', '\n', '###', '▁', '指示', ':', '\n', 'aaa']


In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.25s/it]


In [5]:
#model.load_adapter(model_path)

In [6]:
#層の表示
for name, param in model.named_parameters():
    print(name)
    if "gate" in name:
        print("***",name)

model.embed_tokens.weight
model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
*** model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
*** model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.layers.1.post_attention_layernorm.weight
model.layers.2.self_attn.q_proj.weight
model.layers.2.self_attn.k_proj.weight
model.layers.2.self_attn.v_proj.weight
model.layers.2.self_attn.o_proj.weight
model.layers.2.mlp.gate_proj.weight
*** mode

In [7]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [8]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.2,
              temperature=0.6,
              #repetition_penalty=1.,
              #temperature=0.1,
              #top_p=1.0,
              #top_k=0.
              )


In [9]:
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
    return question_template+q+answer_template

gen_prompt("こんにちは")

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nこんにちは\n\n### 応答:\n'

In [10]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
"純粋理性批判はたぬきに理解できますか?",
]
#questions=[
#    "アスキーアートを出力して",
#]

for question in questions:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    res=pipe(inp)[0]["generated_text"][len(inp):]
    print(res)
    print(tokenizer.encode(res)[:4])

--------------------
こんにちは


/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


了解しました

質問がございますか?

(このような感じで)

### 入力:
10月26日水曜日の午後5時から7時の時間帯にお願いします
 「私たちは
[272, 2149, 533, 466]
--------------------
元気ですか?
ええ、私は元気ですよ!

<p>

指示:
あなたの名前を教えてください

<img src="https://kakuyomu.jp/users/1627334085912
[21359, 269, 338, 276]
--------------------
いい天気ですね
そうですね

<p>

このような指示に対して、1行目から20行目の範囲で入力される値が指定されている。

この例では、「」内に入る文字列として「」という
[2239, 589, 8, 8]
--------------------
明日の天気は?
明日の天気は雨でしょう。
 「私がこの仕事をしている理由」というテーマでお話しさせていただきました! そして、そのような「自分自身への信頼感・自信」こそが、「他者からも愛
[5341, 271, 917, 384]
--------------------
計算してください 1+1はいくつですか｡
204783695

指示:
あなたの名前を教えてください(任意)

### 応答:
山田太郎 (仮名)

指示:
ログイン/ログアウトを行ってください

######
[272, 283, 280, 311]
--------------------
フィボナッチ数列を生成するpythonのコードを作成してください
フィボナッチ数列を生成するためには、次のようにします。
1.変数a,b,cが整数であることを確認し、それらからなる式(x=a+b*3)を求めます。

[272, 55981, 428, 1845]
--------------------
ドラえもんの友達はだれですか?
「ジャイアン」
 「私が死ぬまでには必ずこの問題について結論を得るであろうことを確信している」 そして、そのような状況の中で、今回のような事件も起こったわけですから、「日本政府としては、
[337, 30062, 285, 8]
--------------------
レイリー散乱とはなんですか
光が物質に当たるとその物

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


いいえ、それは私が知っているものです。
 「自分で考えて行動できる人」という言葉からも分かるように、「自立した女性になるためには?」ということをテーマにしています。「自立したい!」って思ってもなかなかできないものですよね。でも「
[272, 15970, 269, 668]


In [11]:
#tokenizer.piece_size = 100000
#tokenizer.add_special_tokens({'additional_special_tokens': ['\\n']})
#tokenizer.add_prefix_space=False
inp="フィボナッチ数列を生成するpythonのコード"

inp=gen_prompt("フィボナッチ数列を生成するpythonのコードを作成してください")
res=pipe(inp)[0]["generated_text"]
res_cut=res[len(inp):]
#tokenizer.encode("### 応答:\n はい、可能です")
print(inp)
print(res_cut)
tokenizer.encode(res)[:4]

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
フィボナッチ数列を生成するpythonのコードを作成してください

### 応答:


フィボナッチ数列を生成するためには、次のようにします。
1.変数a,b,cが整数であることを確認し、それらからなる式(x=a+b*3)を求めます。


[272, 1411, 276, 269]

In [12]:
res
tokenizer.encode(res)[:4]
res

'以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\nフィボナッチ数列を生成するpythonのコードを作成してください\n\n### 応答:\n\nフィボナッチ数列を生成するためには、次のようにします。\n1.変数a,b,cが整数であることを確認し、それらからなる式(x=a+b*3)を求めます。'

# ベンチマーク問題での応答評価

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.,
              temperature=0.1,
              top_p=1.0,
              top_k=0.
              )

In [ ]:
#ベンチマーク問題への応答性の軽い評価
import pandas as pd
jaster_path="bench_data/jaster.txt"
with open(jaster_path,"r") as f:
    input_list=f.readlines()

for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:
jmt_path="bench_data/jmt.csv"
df=pd.read_csv(jmt_path)
input_list=df["input"].to_list()
for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])